[View in Colaboratory](https://colab.research.google.com/github/rutgerhofste/notebook_gis_plotting/blob/master/07_mapboxgl_simple.ipynb)

# Mapbox GL Chloropleth Example with custom data

In [0]:
!pip install mapboxgl geopandas sqlalchemy psycopg2 --quiet

In [0]:
# public token from MapboxGL
token = # copy from mapbox website
os.environ["MAPBOX_ACCESS_TOKEN"] = token

In [0]:
import os
import getpass
import sqlalchemy
import psycopg2
import mapboxgl
import random
import json
import pandas as pd
import geopandas as gpd

In [0]:
# Global Variables
DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
DATABASE_NAME = "database01"

WS_VALUES_INPUT_TABLE = "y2018m06d28_rh_ws_full_range_ols_postgis_30spfaf06_v01_v02"

In [0]:
u = getpass.getpass("Type in your postgresql username: ")

Type in your postgresql username: ··········


In [0]:
p = getpass.getpass("Type in your postgresql password: ")

Type in your postgresql password: ··········


In [0]:
engine = sqlalchemy.create_engine("postgresql://{}:{}@{}:5432/{}".format(u,p,DATABASE_ENDPOINT,DATABASE_NAME))

In [0]:
test_sql = "SELECT * FROM y2018m06d28_rh_ws_full_range_ols_postgis_30spfaf06_v01_v02 WHERE temporal_resolution = 'year' AND year = 2014"

In [0]:
df = pd.read_sql(test_sql,engine)

In [0]:
df.head()

,pfafid_30spfaf06,temporal_resolution,year,month,area_m2_30spfaf06,area_count_30spfaf06,pdomww_m_30spfaf06,pdomwn_m_30spfaf06,pindww_m_30spfaf06,pindwn_m_30spfaf06,...,max_ma10_waterstress_dimensionless_30spfaf06,slope_ma10_waterstress_dimensionless_30spfaf06,intercept_ma10_waterstress_dimensionless_30spfaf06,ols_ma10_waterstress_dimensionless_30spfaf06,avg_ols10_waterstress_dimensionless_30spfaf06,min_ols10_waterstress_dimensionless_30spfaf06,max_ols10_waterstress_dimensionless_30spfaf06,slope_ols10_waterstress_dimensionless_30spfaf06,intercept_ols10_waterstress_dimensionless_30spfaf06,ols_ols10_waterstress_dimensionless_30spfaf06
0,111011,year,2014,12,1.885917e+09,2536,0.005918,0.004587,0.025761,0.010304,...,2.146653,0.016313,-30.388371,2.466601,2.105324,2.146076,2.149802,0.008748,-15.281826,2.337153
1,111012,year,2014,12,2.925797e+09,3921,0.000053,0.000053,0.002746,0.001099,...,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
2,111013,year,2014,12,8.924229e+08,1194,0.000156,0.000156,0.008338,0.003335,...,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
3,111014,year,2014,12,4.206268e+09,5605,0.000031,0.000031,0.000216,0.000086,...,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
4,111015,year,2014,12,1.659706e+10,21873,0.000244,0.000215,0.006480,0.002592,...,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000


In [0]:
df.shape

(16385, 124)

In [0]:
os.environ

environ({'LANG': 'en_US.UTF-8', 'DATALAB_DEBUG': 'true', 'SHELL': '/bin/bash', 'LD_PRELOAD': '/usr/lib/x86_64-linux-gnu/libtcmalloc.so.4', 'JPY_PARENT_PID': '67', 'SHLVL': '1', 'PYTHONPATH': '/env/python', 'OLDPWD': '/', 'HOSTNAME': '55a46eb6fd45', 'CLOUDSDK_CONFIG': '/content/datalab/.config', 'DATALAB_ENV': 'GCE', 'DATALAB_SETTINGS_OVERRIDES': '{"datalabBasePath":"/tun/m/m-167t6un7q9i1z/"}', 'PATH': '/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin', 'PWD': '/', 'GLIBCPP_FORCE_NEW': '1', 'ENV': '/root/.bashrc', 'DEBIAN_FRONTEND': 'noninteractive', 'GLIBCXX_FORCE_NEW': '1', 'HOME': '/content', '_': '/tools/node/bin/forever', 'TERM': 'xterm-color', 'CLICOLOR': '1', 'PAGER': 'cat', 'GIT_PAGER': 'cat', 'MPLBACKEND': 'module://ipykernel.pylab.backend_inline', 'GDAL_DATA': '/usr/local/lib/python3.6/dist-packages/fiona/gdal_data', 'PROJ_LIB': '/usr/local/lib/python3.6/dist-packages/fiona/proj_data', 'MAPBOX_ACCE

In [0]:
data_test = json.loads(df.to_json(orient='records'))

In [0]:
color_stops_test = mapboxgl.utils.create_color_stops([0, 3, 21873], colors='YlOrRd')

In [0]:
# Create the viz
viz = mapboxgl.viz.ChoroplethViz(data = data_test, 
                                  vector_url='mapbox://rutgerhofste.hybas06_v04_V03',
                                  vector_layer_name='hybas06_v04_V03',
                                  vector_join_property='pfaf_id',
                                  data_join_property="pfafid_30spfaf06",
                                  color_property="area_count_30spfaf06",
                                  color_stops=color_stops_test,
                                  line_color = 'rgba(0,0,0,0.05)',
                                  line_width = 0.5,
                                  opacity=0.7,
                                  center=(-95, 45),
                                  zoom=2,
                                  below_layer='waterway-label'
                                  )
viz.show()